In [1]:
!pip install matplotlib



In [2]:
import matplotlib.pyplot as plt

In [3]:
import numpy as np

In [4]:

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [5]:
import os


In [6]:
!pip install Pillow

In [7]:
import PIL

In [8]:
!pip install opendatasets --upgrade --quiet

In [9]:
import opendatasets as od

In [10]:
dataset_url = 'https://www.kaggle.com/alxmamaev/flowers-recognition'

In [11]:
od.download(dataset_url)

Skipping, found downloaded files in ".\flowers-recognition" (use force=True to force download)


In [18]:
data_dir = './flowers-recognition/flowers'

In [19]:
os.listdir(data_dir)

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [20]:
import pathlib
data_dir = pathlib.Path(data_dir)

In [21]:
img_height,img_width=180,180
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 4317 files belonging to 5 classes.
Using 3454 files for training.


In [22]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 4317 files belonging to 5 classes.
Using 863 files for validation.


In [23]:
class_names = train_ds.class_names
print(class_names)

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']


In [24]:
num_classes = 5

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,activation='softmax')
])

In [25]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [26]:
epochs=14
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/14


E:\ANAXONDA\envs\tensorflow_env\lib\site-packages\tensorflow\python\keras\backend.py:4942: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


108/108 [==============================] - 36s 322ms/step - loss: 1.3045 - accuracy: 0.4302 - val_loss: 1.1585 - val_accuracy: 0.4820
Epoch 2/14
108/108 [==============================] - 23s 214ms/step - loss: 1.0205 - accuracy: 0.5938 - val_loss: 0.9950 - val_accuracy: 0.6153
Epoch 3/14
108/108 [==============================] - 22s 207ms/step - loss: 0.8207 - accuracy: 0.6963 - val_loss: 1.0671 - val_accuracy: 0.6107
Epoch 4/14
108/108 [==============================] - 24s 224ms/step - loss: 0.5939 - accuracy: 0.7846 - val_loss: 1.0113 - val_accuracy: 0.6431
Epoch 5/14
108/108 [==============================] - 24s 223ms/step - loss: 0.3834 - accuracy: 0.8564 - val_loss: 1.2967 - val_accuracy: 0.6234
Epoch 6/14
108/108 [==============================] - 24s 216ms/step - loss: 0.2265 - accuracy: 0.9259 - val_loss: 1.5598 - val_accuracy: 0.6269
Epoch 7/14
108/108 [==============================] - 23s 214ms/step - loss: 0.1559 - accuracy: 0.9485 - val_loss: 1.5860 - val_accuracy: 0.6

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [28]:
!pip install gradio --quiet

In [29]:
import gradio as gr

In [30]:
def predict_image(img):
    img_4d=img.reshape(-1,180,180,3)
    prediction=model.predict(img_4d)[0]
    return {class_names[i]: float(prediction[i]) for i in range(5)}

In [ ]:
image = gr.inputs.Image(shape=(180,180))
label = gr.outputs.Label(num_top_classes=5)

gr.Interface(fn=predict_image, inputs=image, outputs=label,interpretation='default').launch(debug='True')

Running locally at: http://127.0.0.1:7860/
To create a public link, set `share=True` in `launch()`.
Interface loading below...
